In [1]:
#PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import numpy as np

#dataset
import torch.utils.data as utils

#For paths
import glob

#imread
from skimage import io, transform

#split dataset
from sklearn.model_selection import train_test_split

#Timestamp
import datetime

In [2]:
root_path = "D:/Downloads/kinect_leap_dataset/acquisitions"
p_id = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10", "P11", "P12", "P13", "P14"]
g_id = ["G1", "G2", "G3", "G4", "G5", "G6", "G7", "G8", "G9", "G10"]

In [3]:
print(root_path + "/P1/G1")
files = glob.glob(root_path + "/P1/G1/" + "*rgb.png")
print(files)
print(len(files))

D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1
['D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\10_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\1_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\2_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\3_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\4_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\5_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\6_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\7_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\8_rgb.png', 'D:/Downloads/kinect_leap_dataset/acquisitions/P1/G1\\9_rgb.png']
10


In [4]:
dataset = []
labels = []

for p in p_id:
    for g in g_id:
        #print(p, g)
        image_names = glob.glob(root_path + "/" +  p + "/" + g + "/" + "*rgb.png")
        for img_path in image_names:
            img = io.imread(img_path)
            img = transform.rescale(img, 1.0 / 20.0)

            #img = np.resize(img,(3, 48, 64))
            img = np.moveaxis(img, [0, 1, 2], [1, 2, 0])
            
            dataset.append(img)
            
            #label 10 will be 0
            tmp = np.zeros(10)
            tmp[int(g[-1])] = 1
            labels.append(tmp)
            #labels.append(int(g[-1]))

C:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
#ffs
dataset = np.array(dataset).astype(float)
labels = np.array(labels).astype(float)

In [6]:
print(type(labels))
print(type(dataset))
X_train, X_test_val, y_train, y_test_val = train_test_split(dataset, labels, test_size=0.2)
print(type(X_train))
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [7]:
dataset = torch.from_numpy(dataset).float()
labels = torch.from_numpy(labels).float()
print(type(dataset))
print(type(labels))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [8]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()

In [9]:
my_dataset = utils.TensorDataset(dataset, labels) # create your dataset
my_dataloader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

In [10]:
my_dataset = utils.TensorDataset(X_train, y_train) # create your dataset
train_loader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

my_dataset = utils.TensorDataset(X_val, y_val) # create your dataset
val_loader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

my_dataset = utils.TensorDataset(X_test, y_test) # create your dataset
test_loader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

In [11]:
#input 640x480
#h=480, w=640
#downscaled by 4

#output 10 classes
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        #self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1 = nn.Linear(16 * 13 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #print(list(x.size()))
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2,2))
        #print(list(x.size()))
        
        #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #print(list(x.size()))
        x = x.view(-1, self.num_flat_features(x))
        
        #print(list(x.size()))
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
model = Net()
print(model)
#if torch.cuda.is_available():
#    net.cuda()

criterion = torch.nn.MSELoss(size_average=False)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

#criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.0)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1872, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
train_loss_history = []
train_acc_history = []
val_acc_history = []
val_loss_history = []
num_epochs = 300
iter_per_epoch = len(train_loader)
#224 lines -> 112 val output for log_nth=10000
#1120000 iter
#log_nth = 16
log_nth = 1000

if torch.cuda.is_available():
    model.cuda()

for epoch in range(num_epochs):  # loop over the dataset multiple times
    for i, (inputs, targets) in enumerate(train_loader, 1):
        inputs, targets = Variable(inputs.float()), Variable(targets.float())
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss_history.append(loss.data.cpu().numpy())
        if log_nth and i % log_nth == 0:
            last_log_nth_losses = train_loss_history[-log_nth:]
            train_loss = np.mean(last_log_nth_losses)
            print('[Iteration %d/%d] TRAIN loss: %.3f' % \
                        (i + epoch * iter_per_epoch,
                         iter_per_epoch * num_epochs,
                         train_loss))
            
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)
    #print(preds)
    #print(targets)
    train_acc = np.mean((preds == target_indices).data.cpu().numpy())
    train_acc_history.append(train_acc)
    if log_nth:
        print('[Epoch %d/%d] TRAIN acc/loss: %.3f/%.3f' % (epoch + 1,
                                                            num_epochs,
                                                            train_acc,
                                                            loss))
        '''_, preds = torch.max(outputs, 1)

        # Only allow images/pixels with label >= 0 e.g. for segmentation
        targets_mask = labels >= 0
        train_acc = np.mean((preds == targets)[targets_mask].data.cpu().numpy())
        train_acc_history.append(train_acc)
        if log_nth:
            print('[Epoch %d/%d] TRAIN acc/loss: %.3f/%.3f' % (epoch + 1,
                                                                   num_epochs,
                                                                   train_acc,
                                                                   train_loss))'''
        
    # VALIDATION
    val_losses = []
    val_scores = []
    model.eval()
    for inputs, targets in val_loader:
        inputs, targets = Variable(inputs), Variable(targets)
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = model.forward(inputs)
        loss = criterion(outputs, targets)
        val_losses.append(loss.data.cpu().numpy())

        _, preds = torch.max(outputs, 1)
        _, target_indices = torch.max(targets, 1)

        scores = np.mean((preds == target_indices).data.cpu().numpy())
        val_scores.append(scores)

    model.train()
    val_acc, val_loss = np.mean(val_scores), np.mean(val_losses)
    val_acc_history.append(val_acc)
    val_loss_history.append(val_loss)
    if log_nth:
        print('[Epoch %d/%d] VAL   acc/loss: %.3f/%.3f' % (epoch + 1,
                                                            num_epochs,
                                                            val_acc,
                                                            val_loss))

print('Finished Training')

[Epoch 1/300] TRAIN acc/loss: 0.300/8.870
[Epoch 1/300] VAL   acc/loss: 0.079/9.055
[Epoch 2/300] TRAIN acc/loss: 0.100/9.027
[Epoch 2/300] VAL   acc/loss: 0.100/9.054
[Epoch 3/300] TRAIN acc/loss: 0.100/8.877
[Epoch 3/300] VAL   acc/loss: 0.164/8.992
[Epoch 4/300] TRAIN acc/loss: 0.300/8.915
[Epoch 4/300] VAL   acc/loss: 0.143/8.961
[Epoch 5/300] TRAIN acc/loss: 0.300/8.733
[Epoch 5/300] VAL   acc/loss: 0.136/8.883
[Epoch 6/300] TRAIN acc/loss: 0.200/8.719
[Epoch 6/300] VAL   acc/loss: 0.164/8.844
[Epoch 7/300] TRAIN acc/loss: 0.300/8.759
[Epoch 7/300] VAL   acc/loss: 0.236/8.741
[Epoch 8/300] TRAIN acc/loss: 0.000/9.148
[Epoch 8/300] VAL   acc/loss: 0.229/8.620
[Epoch 9/300] TRAIN acc/loss: 0.100/8.507
[Epoch 9/300] VAL   acc/loss: 0.214/8.564
[Epoch 10/300] TRAIN acc/loss: 0.100/8.381
[Epoch 10/300] VAL   acc/loss: 0.186/8.522
[Epoch 11/300] TRAIN acc/loss: 0.000/9.392
[Epoch 11/300] VAL   acc/loss: 0.179/8.709
[Epoch 12/300] TRAIN acc/loss: 0.400/7.823
[Epoch 12/300] VAL   acc/loss

[Epoch 96/300] VAL   acc/loss: 0.886/2.791
[Epoch 97/300] TRAIN acc/loss: 1.000/1.152
[Epoch 97/300] VAL   acc/loss: 0.893/2.655
[Epoch 98/300] TRAIN acc/loss: 1.000/1.239
[Epoch 98/300] VAL   acc/loss: 0.900/2.669
[Epoch 99/300] TRAIN acc/loss: 1.000/0.735
[Epoch 99/300] VAL   acc/loss: 0.914/2.529
[Epoch 100/300] TRAIN acc/loss: 1.000/1.287
[Epoch 100/300] VAL   acc/loss: 0.886/2.722
[Epoch 101/300] TRAIN acc/loss: 1.000/0.466
[Epoch 101/300] VAL   acc/loss: 0.900/2.448
[Epoch 102/300] TRAIN acc/loss: 1.000/0.539
[Epoch 102/300] VAL   acc/loss: 0.900/2.441
[Epoch 103/300] TRAIN acc/loss: 1.000/0.604
[Epoch 103/300] VAL   acc/loss: 0.907/2.464
[Epoch 104/300] TRAIN acc/loss: 1.000/0.598
[Epoch 104/300] VAL   acc/loss: 0.893/2.381
[Epoch 105/300] TRAIN acc/loss: 1.000/0.714
[Epoch 105/300] VAL   acc/loss: 0.886/2.531
[Epoch 106/300] TRAIN acc/loss: 1.000/0.998
[Epoch 106/300] VAL   acc/loss: 0.907/2.400
[Epoch 107/300] TRAIN acc/loss: 1.000/0.692
[Epoch 107/300] VAL   acc/loss: 0.900/2

[Epoch 190/300] TRAIN acc/loss: 1.000/0.084
[Epoch 190/300] VAL   acc/loss: 0.907/1.804


In [ ]:
params = list(model.parameters())

In [ ]:
currentDT = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
path = "saved_models/lenet_rgb_" + str(num_epochs) + "_" + currentDT + ".model"
torch.save(model, path)

In [ ]:
#80% -> 10 epochs
#87% -> 50 epochs

scores = []
for inputs, target in test_loader:
    inputs, targets = Variable(inputs), Variable(target)
    if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)
    scores.extend((preds == target_indices).data.cpu().numpy())
    
print('Test set accuracy: %f' % np.mean(scores))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss_history, '-')
#plt.plot(val_loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()

plt.plot(train_acc_history, '-o')
plt.plot(val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
for inputs, target in test_loader:
    inputs, targets = Variable(inputs), Variable(target)
    if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)
    scores.extend((preds == target_indices).data.cpu().numpy())

In [ ]:
for inputs, target in test_loader:
    inputs, targets = Variable(inputs), Variable(target)
    if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)
    
    numpy_inputs = inputs.data.cpu().numpy()
    numpy_outputs = outputs.data.cpu().numpy()
    numpy_targets = targets.data.cpu().numpy()
    
    img = numpy_inputs[0]
    img = img[0, :, :]
    plt.imshow(img)
    plt.title("Predicted: "+str(preds[0].item()) + " Target: "+str(target_indices[0].item()))
    plt.show()
    #currentDT = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    #path = "images/img_" + currentDT + ".png"
    #plt.imsave(path, img.astype(float))